# Dynamical system learning

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

from grid_search import boxplot_feature

## Pendulum

### Running model

In [5]:
#set model_name=simple_pendulum
import run_pendulum

In [ ]:
#set model_name=pendulum_bias
import run_pendulum

In [ ]:
#set model_name=pendulum_batch_10
import run_pendulum

In [ ]:
#set model_name=non_lin_pendulum
import run_pendulum

### Model analysis with noise

In [ ]:
#If no csv files run this (TAKE A LOT OF TIME)
#set data_name = pendulum
import sim

In [10]:
grid_df_pend = pd.read_csv(path_gen + 'pend_df_1000.csv')
grid_df_pend

,Unnamed: 0,noise_std,epoch,batch,min_loss,Linear diff,Quadratic diff,Linear,Quadratic
0,0,0.0,200,10,292459.238782,0.891934,0.010904,[[ 0.312568 0.46302843]\n [-0.02954498 0.8...,[[[-1.8632392e-03 6.6486252e-03]\n [-6.89092...
1,1,0.0,200,50,183203.483711,0.922791,0.011761,[[ 0.46824655 0.3603762 ]\n [-0.00990373 0.6...,[[[-1.3027265e-03 -1.4930618e-03]\n [ 1.51333...
2,2,0.0,200,100,158619.485209,1.084316,0.011797,[[3.2957137e-01 3.3976030e-01]\n [4.5783480e-0...,[[[-1.6540489e-03 -1.5483800e-03]\n [ 1.45801...
3,3,0.0,500,10,20804.140871,0.471322,0.011724,[[ 0.7516491 0.600944 ]\n [-0.02061452 0.9...,[[[-7.0437242e-04 -1.5669891e-03]\n [ 1.43941...
4,4,0.0,500,50,48880.665974,0.575118,0.011697,[[ 0.6802195 0.56806266]\n [-0.01024296 0.7...,[[[-7.7941682e-04 -1.4981214e-03]\n [ 1.50827...
...,...,...,...,...,...,...,...,...,...
2155,2155,0.5,500,50,68173.705838,0.641460,0.006621,[[ 0.66178364 0.5119639 ]\n [-0.01027527 0.7...,[[[-8.2368503e-04 3.7427575e-03]\n [-3.72978...
2156,2156,0.5,500,100,92205.039196,0.849417,0.006667,[[ 0.49854758 0.45091832]\n [-0.00989506 0.5...,[[[-1.2023000e-03 3.6876600e-03]\n [-3.78488...
2157,2157,0.5,1000,10,7173.663532,0.350596,0.006593,[[ 0.83989424 0.6882254 ]\n [-0.01510846 0.9...,[[[-4.3708290e-04 3.7060706e-03]\n [-3.76646...
2158,2158,0.5,1000,50,11456.624531,0.342147,0.006559,[[ 0.8384697 0.71458554]\n [-0.01012892 0.9...,[[[-3.9277560e-04 3.7392010e-03]\n [-3.73334...


In [ ]:
boxplot_feature(data=grid_df_pend, fixed_feat1='epoch', val_feat1=1000, fixed_feat2='batch', val_feat2=10, feat_x='noise_std', feat_y='min_loss');
boxplot_feature(data=grid_df_pend, fixed_feat1='epoch', val_feat1=1000, fixed_feat2='batch', val_feat2=10, feat_x='noise_std', feat_y='Linear diff');
boxplot_feature(data=grid_df_pend, fixed_feat1='epoch', val_feat1=1000, fixed_feat2='batch', val_feat2=10, feat_x='noise_std', feat_y='Quadratic diff')

In [ ]:
boxplot_feature(data=grid_df_pend, fixed_feat1='noise_std', val_feat1=0, fixed_feat2='batch', val_feat2=10, feat_x='epoch', feat_y='min_loss');
boxplot_feature(data=grid_df_pend, fixed_feat1='noise_std', val_feat1=0, fixed_feat2='batch', val_feat2=10, feat_x='epoch', feat_y='Linear diff');
boxplot_feature(data=grid_df_pend, fixed_feat1='noise_std', val_feat1=0, fixed_feat2='batch', val_feat2=10, feat_x='epoch', feat_y='Quadratic diff')

In [ ]:
boxplot_feature(data=grid_df_pend, fixed_feat1='noise_std', val_feat1=0, fixed_feat2='epoch', val_feat2=1000, feat_x='batch', feat_y='min_loss');
boxplot_feature(data=grid_df_pend, fixed_feat1='noise_std', val_feat1=0, fixed_feat2='epoch', val_feat2=1000, feat_x='batch', feat_y='Linear diff');
boxplot_feature(data=grid_df_pend, fixed_feat1='noise_std', val_feat1=0, fixed_feat2='epoch', val_feat2=1000, feat_x='batch', feat_y='Quadratic diff')

## Lorenz attractor

In [ ]:
#set model_name = non_linear
import run_lorenz

In [ ]:
#set model_name = non_linear_bias
import run_lorenz

### Model analysis with noise

In [ ]:
#If no csv files run this (TAKE A LOT OF TIME)
#set data_name = lorenz
import sim

In [9]:
grid_df_lorenz_1000 = pd.read_csv(path_gen + 'lorenz_df_1000.csv')
grid_df_lorenz_1000

,Unnamed: 0,noise_std,epoch,batch,min_loss,Linear diff,Quadratic diff,Linear,Quadratic
0,0,0.0,200,10,415.235054,33.025593,0.133424,[[ 0.2660012 0.34516168 0.00481499]\n [ 0.3...,[[[-7.09583797e-03 7.83991534e-03 8.44447222...
1,1,0.0,200,50,66.232870,32.987263,0.116629,[[ 0.34060892 0.4269496 0.06105396]\n [ 0.3...,[[[-0.00148063 0.00482754 0.00431253]\n [-0...
2,2,0.0,200,100,41.975578,33.012535,0.115472,[[ 0.33691448 0.36362654 0.06322888]\n [ 0.3...,[[[-0.00131757 0.00499924 0.00434631]\n [-0...
3,3,0.0,500,10,146.536859,33.020958,0.114964,[[ 0.42286003 0.4058006 0.057808 ]\n [ 0.3...,[[[-0.00100963 0.00493515 0.0034609 ]\n [-0...
4,4,0.0,500,50,33.612342,32.952980,0.125419,[[ 0.4109212 0.5945694 0.05995847]\n [ 0.3...,[[[-0.00160085 0.00515679 0.00358583]\n [-0...
...,...,...,...,...,...,...,...,...,...
2155,2155,0.5,500,50,36.860352,33.004920,0.117391,[[ 0.44619182 0.5621652 0.05610231]\n [ 0.3...,[[[-0.00057928 -0.00202912 0.00598058]\n [ 0...
2156,2156,0.5,500,100,23.821163,33.006050,0.121696,[[ 0.43050918 0.53700244 0.05877204]\n [ 0.3...,[[[-1.1153529e-04 -2.0302050e-03 6.1354749e-0...
2157,2157,0.5,1000,10,65.516961,33.094006,0.079342,[[ 0.5034227 0.44366065 0.06777187]\n [ 0.3...,[[[-0.00109634 -0.00182253 0.00533386]\n [ 0...
2158,2158,0.5,1000,50,22.808503,33.038460,0.104724,[[ 0.5035903 0.5947599 0.06572342]\n [ 0.3...,[[[-1.1299325e-03 -1.8228509e-03 5.3273714e-0...


In [ ]:
boxplot_feature(data=grid_df_lorenz_1000, fixed_feat1='epoch', val_feat1=1000, fixed_feat2='batch', val_feat2=10, feat_x='noise_std', feat_y='min_loss');
boxplot_feature(data=grid_df_lorenz_1000, fixed_feat1='epoch', val_feat1=1000, fixed_feat2='batch', val_feat2=10, feat_x='noise_std', feat_y='Linear diff');
boxplot_feature(data=grid_df_lorenz_1000, fixed_feat1='epoch', val_feat1=1000, fixed_feat2='batch', val_feat2=10, feat_x='noise_std', feat_y='Quadratic diff')

In [ ]:
boxplot_feature(data=grid_df_lorenz_1000, fixed_feat1='noise_std', val_feat1=0, fixed_feat2='batch', val_feat2=10, feat_x='epoch', feat_y='min_loss');
boxplot_feature(data=grid_df_lorenz_1000, fixed_feat1='noise_std', val_feat1=0, fixed_feat2='batch', val_feat2=10, feat_x='epoch', feat_y='Linear diff');
boxplot_feature(data=grid_df_lorenz_1000, fixed_feat1='noise_std', val_feat1=0, fixed_feat2='batch', val_feat2=10, feat_x='epoch', feat_y='Quadratic diff')

In [ ]:
boxplot_feature(data=grid_df_lorenz_1000, fixed_feat1='noise_std', val_feat1=0, fixed_feat2='epoch', val_feat2=1000, feat_x='batch', feat_y='min_loss');
boxplot_feature(data=grid_df_lorenz_1000, fixed_feat1='noise_std', val_feat1=0, fixed_feat2='epoch', val_feat2=1000, feat_x='batch', feat_y='Linear diff');
boxplot_feature(data=grid_df_lorenz_1000, fixed_feat1='noise_std', val_feat1=0, fixed_feat2='epoch', val_feat2=1000, feat_x='batch', feat_y='Quadratic diff')